# [DeepID人脸识别算法之三代](http://www.cnblogs.com/mfrbuaa/p/5394742.html)

In [111]:
import gluonbook as gb
import numpy as np
from mxnet import autograd, gluon, nd
from mxnet.gluon import nn
import mxnet as mx

class VGG(nn.Block):
    
    def __init__(self, n_classes, **kwargs):
        super().__init__(**kwargs)
        self.n_classes = n_classes
        self.conv_arch = ((1,64), (1,128), (2,256), (2,512), (2,512))
        self.dense_arch = ((2, 256), (1, 128))
        self._vgg_dense()
        
    def _vgg_block(self, num_convs, num_channels):
        blk = nn.Sequential()
        for _ in range(num_convs):
            blk.add(nn.Conv2D(num_channels, kernel_size=3, padding=1, activation='relu'))
        blk.add(nn.MaxPool2D(pool_size=2, strides=2))
        return blk
    
    def _dense_block(self, num_convs, num_channels):
        blk = nn.Sequential()
        for _ in range(num_convs):
            blk.add(nn.Conv2D(num_channels, kernel_size=1, padding=1, activation='relu'))
        return blk
        
    def _vgg_dense(self):
        # 卷积层部分
        self.features = nn.Sequential()
        for (num_convs, num_channels) in self.conv_arch:
            self.features.add(self._vgg_block(num_convs, num_channels))
        
        self.dense = nn.Sequential()
        for (num_convs, num_channels) in self.dense_arch:
            self.dense.add(self._dense_block(num_convs, num_channels))
        self.dense.add(
            nn.GlobalAvgPool2D(),
            nn.Dropout(.5),
            nn.Dense(self.n_classes))
    
    def forward(self, x):
        x = self.features(x)
        x = self.dense(x)
        return x

net = VGG(10, prefix='vgg')

net

VGG(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2D(None -> 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2D(None -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2D(None -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    )
    (3): Sequential(
      (0): Conv2D(None -> 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2D(None -> 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
    )
    (4): Sequential(
      (0): Conv2D(Non

In [ ]:
lr = 0.05
ctx = gb.try_gpu()
net.initialize(ctx=ctx, init=mx.init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_data, test_data = gb.load_data_fashion_mnist(batch_size=8, resize=224)
loss = gluon.loss.SoftmaxCrossEntropyLoss()
gb.train(train_data, test_data, net, loss, trainer, ctx, num_epochs=3)